# RAG with Ollama


> Note: Before proceeding further you need to download and run Ollama: https://ollama.com/.

This notebook demonstrates how to use the `rag` model with Ollama. 
Source: https://dev.to/mohsin_rashid_13537f11a91/rag-with-ollama-1049

* Import Libraries

In [1]:
from langchain_ollama import OllamaLLM
from langchain_ollama.embeddings import OllamaEmbeddings

from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.chains import create_retrieval_chain
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain
from uuid import uuid4

from datasets import load_dataset
import pandas as pd

From the command line:

* Llama model :

```bash
ollama run llama3.2
```

* Mistral7B model :

```bash
ollama run mistral
```

* Mistral NeMo (12B)

```bash
ollama run mistral-nemo
```



* Loading The LLM (Language Model)

In [2]:
#model_name = "mistral"
model_name = "llama3.2"

llm = OllamaLLM(model=model_name, base_url="http://127.0.0.1:11434")

* Setting Ollama Embeddings

In [3]:
embed_model = OllamaEmbeddings(
    model=model_name,
    base_url='http://127.0.0.1:11434'
)

* Loading Text

In [4]:
ds = load_dataset("GEODE/GeoEDdA")

In [5]:
#dfs = []
#for key in ds.keys():
#    dfs.append(pd.DataFrame({'dataset':key, 'text':ds[key]['text'], 'meta':ds[key]['meta']}))
#df = pd.concat(dfs, ignore_index=True)

#text = " ".join(df.text.to_list())

In [18]:
nb_docs = 100
head = False
chunk_size = 300 # can be None
chunk_overlap = 50

In [19]:
ds['test']['text'][:nb_docs]

['COMPIEGNE, (Géog. mod.) ville de France, dans l\'île de France. Long. 20d. 29\'. 41". lat. 49d. 24\'. 59".',
 "HILPERHAUSEN, (Géog.) ville d'Allemagne en Franconie, sur la Werra, au comté de Henneberg, entre Cobourg & Smalcalde ; elle appartient à une branche de la maison de Saxe-Gotha. Long. 28. 15. lat. 50. 35. (D. J.)",
 "Patane ou Patany, (Géog. mod.) royaume des Indes dans la presqu'île de Malaca, sur la côte orientale, entre le royaume de Siam & de Paha. Les habitans sont en partie mahométans & en partie payens. Les Chinois font avec eux un grand commerce ; on n'y distingue que deux saisons, l'hiver & l'été; l'hiver dure pendant les mois de Novembre, Décembre & Janvier, pendant lesquels il pleut sans cesse. Les bois sont remplis d'éléphans, de sangliers & de guenons. Le royaume, dit Gervaise, releve du roi de Siam, & est gouverné par une reine qui ne peut se marier, mais qui peut avoir des amans tant qu'elle veut. La lubricité des femmes y est si grande, que les hommes sont con

* Option1: Splitting Text into Chunks

In [8]:
#chunks = ["ILLESCAS, (Géog.) petite ville d'Espagne, dans la nouvelle Castille, à six lieues au sud de Madrid.", 
#          "MULHAUSEN, (Géog.) ville impériale d'Allemagne, dans la Thuringe, sous la protection de l'électeur de Saxe, ce qui fait qu'elle est rangée parmi les villes de basse-Saxe ; elle a essuyé bien des calamités en divers tems. Henri le Lion la prit d'assaut en 1181, & la brûla. En 1366 un tremblement de terre en renversa la plus grande partie ; en 1442 un incendie ne lui fut guère moins funeste ; en 1525 elle fut assiegée par l'électeur de Saxe & le landgrave de Hesse, à cause des paysans révoltés qui s'en étoient emparés ; enfin après la paix de Westphalie, les divers partis l'ont ravagée tour-à-tour. Elle est située dans un pays fertile, sur la riviere d'Unstruth, à 5 milles de Nordhausen, 6 N. E. d'Eysenach, 10 N. O. d'Erford, 14 S. O. de Cassel. Long. 28. 14. lat. 51. 13. (D. J.)"
#            ]

In [9]:
chunks = []
#for doc in df.text.to_list():
for doc in ds['test']['text'][:nb_docs]:
    if len(doc) > 512:
        chunks.extend(RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128).split_text(doc))
    else:
        chunks.append(doc)

In [10]:
len(chunks)

112

* Creating a Vector Store (Chroma) from Text

In [11]:
#vector_store = Chroma.from_texts(chunks, embed_model)

* Option2: Splitting Text into Documents

In [20]:
docs = []
if chunk_size is not None:
    j = 0
    for index, chunk in enumerate(ds['test']['text'][:nb_docs]):
        if len(chunk) > chunk_size:
            chunks = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap).split_text(chunk)
            for i, c in enumerate(chunks):
                if head:
                    ch = ds['test']['meta'][index]["head"] + ", " + c if i > 0 else c
                else:
                    ch = c
                docs.append(Document(page_content=ch, metadata={"head": ds['test']['meta'][index]["head"]}, id=j))
                j += 1
        else:
            docs.append(Document(page_content=chunk, metadata={"head": ds['test']['meta'][index]["head"]}, id=j))
            j += 1
else:
    docs = [Document(page_content=chunk, metadata={"head": ds['test']['meta'][index]["head"]}, id=index) for index, chunk in enumerate(ds['test']['text'][:nb_docs])]
docs

[Document(id='0', metadata={'head': 'COMPIEGNE'}, page_content='COMPIEGNE, (Géog. mod.) ville de France, dans l\'île de France. Long. 20d. 29\'. 41". lat. 49d. 24\'. 59".'),
 Document(id='1', metadata={'head': 'HILPERHAUSEN'}, page_content="HILPERHAUSEN, (Géog.) ville d'Allemagne en Franconie, sur la Werra, au comté de Henneberg, entre Cobourg & Smalcalde ; elle appartient à une branche de la maison de Saxe-Gotha. Long. 28. 15. lat. 50. 35. (D. J.)"),
 Document(id='2', metadata={'head': 'Patane ou Patany'}, page_content="Patane ou Patany, (Géog. mod.) royaume des Indes dans la presqu'île de Malaca, sur la côte orientale, entre le royaume de Siam & de Paha. Les habitans sont en partie mahométans & en partie payens. Les Chinois font avec eux un grand commerce ; on n'y distingue que deux saisons, l'hiver & l'été;"),
 Document(id='3', metadata={'head': 'Patane ou Patany'}, page_content="n'y distingue que deux saisons, l'hiver & l'été; l'hiver dure pendant les mois de Novembre, Décembre & J

* Creating a Vector Store (Chroma) from Documents and persist the vector store to disk

In [ ]:
#vector_store.reset_collection()

# list of existing collections
# "nb_docs_100_chunk-size_300_head_True"
# "nb_docs_100_chunk-size_300_head_False"
# "nb_docs_100_chunk-size_None_head_False"


In [33]:
#collection_name = "nb_docs_"+str(nb_docs)+"_chunk-size_"+str(chunk_size)+"_head_"+str(head)

#collection_name = "nb_docs_100_chunk-size_300_head_True"
collection_name = "nb_docs_100_chunk-size_300_head_False"
#collection_name = "nb_docs_100_chunk-size_None_head_False"

vector_store = Chroma(
    collection_name=collection_name,
    embedding_function=embed_model,
    persist_directory="./chroma_langchain_db", 
)

* Add documents to the collection:

In [ ]:
#uuids = [str(uuid4()) for _ in range(len(docs))]

#vector_store.add_documents(documents=docs, ids=uuids)

* Test a similarity search with the vector store

In [34]:
results = vector_store.similarity_search_with_score(query="DUSSELDORP",k=1)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

* [SIM=1.322724] de la Rhétie. Long. 27. 35. latit. 47. 36. (D. J.) [{'head': 'Wangen'}]


In [ ]:
results = vector_store.similarity_search_with_score(query="Dans quel pays se situe DUSSELDORP ?",k=1)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

* Creating a Retriever

In [ ]:
retriever = vector_store.as_retriever()

* Creating a Retrieval Chain

In [ ]:
chain = create_retrieval_chain(combine_docs_chain=llm,retriever=retriever)

* Retrieval-QA Chat Prompt

In [ ]:
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

* Combining Documents

In [ ]:
combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)

* Final Retrieval Chain

In [ ]:
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)    

* Invoking the Retrieval Chain

In [ ]:
response = retrieval_chain.invoke({"input": "Dans quel pays se situe DUSSELDORP ?"})
print(response['answer'])

In [ ]:
response = retrieval_chain.invoke({"input": "Quel est la localisation de Patany (latitude / longitude) ?"})
print(response['answer'])

## TODO: 

* faire un test des mêmes requêtes mais avec les chunks sans la concaténation du titre d'article
* faire un test des mêmes requêtes mais avec les chunks sans limite de taille
* ...


In [ ]:
vector_store.delete_collection()